In [1]:
from pynndescent import NNDescent
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import numpy as np
from keras.datasets import mnist
from time import perf_counter
from ast import literal_eval

In [2]:
name = 'glove-100-angular'
nameFull = name +'-true-labels.xlsx'
datasetTrainImages, datasetTestImages, _ = get_ann_benchmark_data(name)

<string>:57: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


trainDataset :  (1183514, 100)
testDataset :  (10000, 100)


<string>:58: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
<string>:59: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


***Create faiss***

In [10]:
def createIndex(indexMethod, datasetImages):
    time_start = perf_counter()
    index = indexMethod(datasetImages, metric="euclidean")
    time_end = perf_counter()
    totalTime = (time_end - time_start)
    print(f'Took {totalTime:.3f} seconds')
    return (index, totalTime)
(indexedStruct, time) = createIndex(NNDescent, datasetTrainImages)

Took 183.522 seconds


In [11]:
# (min, max) = createIndexNumerous(createIndex, AnnoyIndex, datasetImages, 10)
# print('min : ', min, '\n','max : ', max,)

In [12]:
indexes = []
distances = []
def measureTime(par, indexes, distances, datasetImages):
    totalTime = 0
    for i in range(par) : 
        xq = datasetImages[i:i+1].astype('float32') # Use the first image as the query vector
        time_start = perf_counter()
        index, distance = indexedStruct.query(xq, k=100)
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        indexes.append(index[0])
        distances.append(distance[0])
        # distances.append(np.sqrt(distance[0]))
        # indexes.append(index[0])
    # report the duration
    print(f'Took {totalTime:.3f} seconds')
measureTime(1000, indexes, distances, datasetTestImages)

Took 120.816 seconds


In [13]:
# (min, max) = measureTimeNumerous(measureTime, 10)
# print('min : ', min, '\n','max : ', max,)

In [14]:
indexes = np.array(indexes)
distances = np.round(np.array(distances).astype(float), 4)

In [15]:
print('indexes : ', indexes.shape)
print('distances : ', distances.shape)

indexes :  (1000, 100)
distances :  (1000, 100)


In [16]:
path = '../datasets/'+nameFull
(trueIndexes, trueDistances) = readDB(path)

trueIndexes :  (1000, 100)
trueDistances :  (1000, 100)


In [17]:
compareFirstTen(indexes, distances, trueIndexes, trueDistances)

660281 || 660281
3.8792 || 3.8792
93438 || 93438
3.8838 || 3.8838
1133489 || 1133489
3.9021 || 3.9021
97478 || 97478
3.9037 || 3.9037
995275 || 995275
3.9294 || 3.9294
207076 || 207076
3.9378 || 3.9378
989330 || 989330
3.9523 || 3.9523
846101 || 846101
3.955 || 3.955
242797 || 242797
3.9568 || 3.9568
402216 || 402216
3.9697 || 3.9697


In [18]:
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.01)
calculateRecallAverage(indexes, distances, trueIndexes, trueDistances, 1.1)

0.2974
0.9915
0.9999


In [19]:
print(indexes[0]) # will find the 100 nearest neighbors

[ 660281   93438 1133489   97478  995275  207076  989330  846101  242797
  402216  671078  926741  256447  914908  229951  956338  859959   56568
  647843  987056  900915  963392  100544  861914 1025986   80231  134899
  294110  716135  452141  264602  499677  834699   17932  528570  392021
  482291  660541  368655  583340 1014707   51821  817757  939324  147983
  530481 1136921  979282   96651  430077  645035  924263  689945  292551
  876802  796611 1034304   94664  599119  639351  870693  904716  455463
  209031  100206  402905  186281 1134185 1150727  896732  135950   82685
  867359  783290  445936   21236  273527 1169856  988080  680202  444778
  491578 1116758 1127655  427738  486364  733580  816741  699013  380601
 1051872  521115  703678  416553  290452 1059144  751282  636153 1182920
  327720]


In [20]:
# draw_mnist(indexes, distances, datasetTrainImages)